In [1]:
import sys
# from graph_dataset.dataset import GraphDataset
from rgcn_dataset import GraphDataset
from model.bipartite2 import GNNPolicy

from utilities import train_cl as train
import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
import gurobipy as gp
import json
from torch_geometric.data import HeteroData

In [2]:
#set folder
instance = '80_c11_based_8_std'
model = 'bi_cl_2'
uc_model = '3bin'

model_save_path = f"./model_save/local_search_{uc_model}_{instance.split('_')[0]}_{model}/"
log_save_path =   f"./model_save/local_search_{uc_model}_{instance.split('_')[0]}_{model}_log/"
os.makedirs(model_save_path, exist_ok=True)
os.makedirs(log_save_path, exist_ok=True)

log_file = open(f'{log_save_path}train.log', 'wb')

In [3]:
LEARNING_RATE = 1e-4
NB_EPOCHS =1000
BATCH_SIZE = 1
NUM_WORKERS = 0
ETA_MIN=1e-5

In [4]:
train_data = GraphDataset(f'./datasets/{instance}/small_range_local_search_tripartite/train/{uc_model}/')
valid_data = GraphDataset(f'./datasets/{instance}/small_range_local_search_tripartite/valid/{uc_model}/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
if os.path.exists(model_save_path+'model_best_con.pth'):
    state_dict = torch.load(model_save_path+'model_best_con.pth')
    PredictModel.load_state_dict(state_dict, strict=False)
    print("载入模型成功")

In [6]:
def init_weights(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
PredictModel.apply(init_weights)
print('init')

init


In [7]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS//10, eta_min=ETA_MIN)
best_val_loss = 3
cur_epoch = 0


In [8]:
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss = train(PredictModel, train_iter, DEVICE, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.8f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, DEVICE, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.8f}")
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print("best_loss",best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best_con.pth')
    if epoch%2==0:
        torch.save(PredictModel.state_dict(),model_save_path+f'model_{epoch}.pth')
    if cur_epoch>100:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 2.89999700	Epoch 0 Valid loss: 2.92002575
best_loss 2.9200257530177596
Epoch 1 Train loss: 2.89918558	Epoch 1 Valid loss: 2.92022861
Epoch 2 Train loss: 2.89909125	Epoch 2 Valid loss: 2.91999156
best_loss 2.9199915631844178
Epoch 3 Train loss: 2.89903776	Epoch 3 Valid loss: 2.91996425
best_loss 2.9199642479854777
Epoch 4 Train loss: 2.89903531	Epoch 4 Valid loss: 2.91993957
best_loss 2.91993957314178
Epoch 5 Train loss: 2.89898416	Epoch 5 Valid loss: 2.92002687
Epoch 6 Train loss: 2.89899130	Epoch 6 Valid loss: 2.91992577
best_loss 2.9199257689670923
Epoch 7 Train loss: 2.89898465	Epoch 7 Valid loss: 2.92010475
Epoch 8 Train loss: 2.89899062	Epoch 8 Valid loss: 2.92012517
Epoch 9 Train loss: 2.89900291	Epoch 9 Valid loss: 2.91994006
Epoch 10 Train loss: 2.89898623	Epoch 10 Valid loss: 2.91992729
Epoch 11 Train loss: 2.89897387	Epoch 11 Valid loss: 2.92003197
Epoch 12 Train loss: 2.89900090	Epoch 12 Valid loss: 2.91991680
best_loss 2.9199168005128846
Epoch 13 Train l